### Universidad Nacional de Lujan - Bases de Datos Masivas (11088) - Cavasin Nicolas #143501

# TP06: Frameworks de procesamiento distribuido

## Hadoop MapReduce:
El archivo *ventas.txt* posee las ventas de una empresa con los siguientes datos: *id_vendedor*, *id_coordinador*, *cantidad_de_productos_vendidos*, *cantidad_de_dinero*.

Genere un esquema bajo el paradigma *MapReduce* para resolver las siguientes consignas:

- Produzca un mapper y un reducer para responder cuál es el bonus obtenido por cada vendedor siendo que cada vendedor obtiene el 3% del total del dinero vendido.

- Produzca un mapper y un reducer para obtener la cantidad de productos vendidos por cada vendedor, agrupado por coordinador.

**Nota:** para facilitar la lectura se ha comentado la impresion por consola. Modificar a gusto.



In [49]:
!wget https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP06/data/ventas.txt

--2020-12-04 03:09:17--  https://raw.githubusercontent.com/bdm-unlu/2020/master/TPs/TP06/data/ventas.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3705191 (3.5M) [text/plain]
Saving to: ‘ventas.txt.1’

ventas.txt.1        100%[===================>]   3.53M  --.-KB/s    in 0.1s    

2020-12-04 03:09:18 (30.5 MB/s) - ‘ventas.txt.1’ saved [3705191/3705191]



Para emular el funcionamiento de Hadoop, se utilizarán 3 scripts:

1. ``mapper.py``: 
    - Realiza la transformación al formato deseado ``<key, value>`` quedando ``<id_vendedor, cantidad_de_dinero>``.
    - Escribe el resultado a un archivo *01-mapped_a.txt* en formato *.tsv* para que lo consuma el siguiente script.

2. ``sorter.py``: 
    - Ordena la salida del ``script_mapper`` por *id_vendedor*.
    - Escribe un nuevo archivo *02-sorted_b.txt* en formato *.tsv*.

3. ``reducer.py``:
    - Consume la salida del ``script_sorter``.
    - Acumula y calcula el 3% del total de dinero que le corresponde a cada *id_vendedor*.
    - Escribe los resultados en nuevo archivo *03-reduced_b.txt* en formato *.tsv*.

___

# Paso 1 - Mapping:

In [61]:
# Abro el archivo ventas
with open("ventas.txt") as file_hdfs:

    # Creo el archivo mapper.txt
    with open('01-mapped_a.txt', 'w') as map_file:

        # Leo cada linea del archivo
        for line in file_hdfs:

            # Se eliminan los espacios en blanco iniciales y finales
            line = line.strip()

            # Separo la linea en palabras y obtengo una lista 
            words = line.split()

            # Imprimo el id_vendedor y cantidad_de_dinero
            map_file.write(f'{words[0]}\t{words[3]}\n')

# 'with' cierra automaticamente todos los archivos

# Paso 2 - Sorting:

In [62]:
# Abro el archivo mapped
with open('01-mapped_a.txt', 'r') as map_file:

    # Creo el archivo sorted
    with open('02-sorted_a.txt', 'w') as sort_file:

        # Ordeno y escribo el nuevo archivo ordenado
        for line in sorted(map_file):
            sort_file.write(line)

# Paso 3 - Reducing:

In [63]:
# Referencias
vendedor_actual = None
dinero_actual = 0

# Abro el archivo ordenado por id_vendedor
with open('02-sorted_a.txt', 'r') as sort_file:

    # Creo el archivo reducer
    with open('03-reduced_a.txt', 'w') as red_file:

        # Por cada linea del sorter
        for line in sort_file:

            # Obtengo el <key, value> de cada linea 
            vendedor, dinero = line.split('\t')

            # Convierto dinero a float
            dinero = float(dinero)

            # Si no hubo un cambio de vendedor acumulo
            if vendedor == vendedor_actual:
                dinero_actual += dinero
            else:
                # Si hubo un cambio de vendedor imprimo y actualizo

                if vendedor:
                    # Calculo el bono
                    bono = dinero_actual * 0.03

                    # Informo por consola - ELIMINAR COMENTARIO PARA VISUALIZAR
                    #print(vendedor_actual, '\t', bono)

                    # Escribo tambien el archivo reduced
                    red_file.write(f'{vendedor_actual}\t{bono}\n')

                    # Modifico los 'actuales'
                    vendedor_actual = vendedor
                    dinero_actual = dinero


# Agrego la ultima linea procesada
bono = (dinero_actual + dinero)* 0.03

# ELIMINAR COMENTARIO PARA VISUALIZAR
# print(vendedor_actual, '\t', bono)

with open('03-reduced_a.txt', 'a') as red_file:
    red_file.write(f'{vendedor_actual}\t{bono}\n')

___

A continuación se realiza el mismo proceso pero para el punto b:

1. ``mapper.py``: 
    - Realiza la transformación al formato deseado ``<key, value>`` quedando ``<[id_coordinador, id_vendedor], cantidad_de_productos_vendidos>``.
    - Escribe el resultado a un archivo *01-mapped_b.txt* en formato *.tsv* para que lo consuma el siguiente script.

2. ``sorter.py``: 
    - Ordena la salida del ``script_mapper`` por *id_coordinador* + *id_vendedor*.
    - Escribe un nuevo archivo *02-sorted_b.txt* en formato *.tsv*.

3. ``reducer.py``:
    - Consume la salida del ``script_sorter``.
    - Calcula el total de productos vendidos agrupados por coordinador y vendedor.
    - Escribe los resultados en nuevo archivo *03-reduced_b.txt* en formato *.tsv*.

In [64]:
# Abro el archivo ventas
with open("ventas.txt") as file_hdfs:

    # Creo el archivo mapper.txt
    with open('01-mapped_b.txt', 'w') as map_file:

        # Leo cada linea del archivo
        for line in file_hdfs:

            # Se eliminan los espacios en blanco iniciales y finales
            line = line.strip()

            # Separo la linea en palabras y obtengo una lista 
            words = line.split()

            # Imprimo el id_coordinador, id_vendedor y cantidad_de_productos_vendidos
            map_file.write(f'{words[1]}\t{words[0]}\t{words[2]}\n')

# 'with' cierra automaticamente todos los archivos

# Abro el archivo mapped
with open('01-mapped_b.txt', 'r') as map_file:

    # Creo el archivo sorted
    with open('02-sorted_b.txt', 'w') as sorted_file:

        # Ordeno y escribo el nuevo archivo ordenado
        for line in sorted(map_file):
            sorted_file.write(line)

# Referencias
coordinador_actual = None
vendedor_actual = None
cantidad_actual = 0

# Abro el archivo ordenado por id_vendedor
with open('02-sorted_b.txt', 'r') as sorted_file:

    # Creo el archivo reducer
    with open('03-reduced_b.txt', 'w') as red_file:

        # Por cada linea del sorted
        for line in sorted_file:

            # Obtengo el <key, value> de cada linea 
            coordinador, vendedor, cantidad = line.split('\t')

            # Convierto de string a integer
            cantidad = int(dinero)

            # Si no hubo un cambio de clave, acumulo
            if coordinador == coordinador_actual and vendedor == vendedor_actual:
                cantidad_actual += cantidad
            else:
                # Si hubo un cambio de coordinador/vendedor imprimo y actualizo

                # Informo por consola - ELIMINAR COMENTARIO PARA VISUALIZAR
                # print(coordinador_actual, '\t', vendedor_actual, '\t', cantidad_actual)

                # Escribo tambien el archivo reduced
                red_file.write(f'{coordinador_actual}\t{vendedor_actual}\t{cantidad_actual}\n')

                # Modifico los 'actuales'
                coordinador_actual = coordinador
                vendedor_actual = vendedor
                cantidad_actual = cantidad

# Agrego la ultima linea procesada - ELIMINAR COMENTARIO PARA VISUALIZAR
# print(coordinador_actual, '\t', vendedor_actual, '\t', cantidad_actual+cantidad)

with open('03-reduced_b.txt', 'a') as red_file:
    red_file.write(f'{coordinador_actual}\t{vendedor_actual}\t{cantidad_actual+cantidad}\n')

## Apache Spark con PySpark:
Resuelva el ejercicio anterior con PySpark.


In [66]:
# Instalo pyspark y configuro el entorno
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [67]:
# Importo Spark
from pyspark import SparkConf, SparkContext

# Seteo el master al entorno local y defino el nombre de la app para identificarla
conf = SparkConf().setMaster("local").setAppName("Bono vendedores")

# Inicializo el Spark Context
sc = SparkContext(conf = conf)

# Verifico inicializacion
sc

<SparkContext master=local appName=Bono vendedores>

In [68]:
# Leo el archivo de ventas y separo cada valor 
rdd_ventas = sc.textFile("ventas.txt").\
                map(lambda line: line.split("\t"))

# Muestro la primer linea del archivo
print(f'Primer linea del archivo: {rdd_ventas.first()}')
print(f'Ocurrencias en archivo: {rdd_ventas.count()}.\n')

# Mapeo seleccionando id_vendedor y cantidad_de_dinero (columnas 0 y 3)
# Ademas convierto dinero a float y ordeno por id_vendedor
rdd_vendedor_dinero = rdd_ventas.\
                        map(lambda values: (values[0], float(values[3]))).\
                        sortByKey()

# Muestro el primer mapeo
print(f'Primer elemento del mapper: <{rdd_vendedor_dinero.first()}>')
print(f'Ocurrencias en mapper: {rdd_vendedor_dinero.count()}.\n')

# Ahora reduzco por id_vendedor y acumulo el dinero
rdd_vendedor_dinero = rdd_vendedor_dinero.reduceByKey(lambda id, dinero: id +dinero)

# Mapeo nuevamente para poder aplicar una multiplicacion y asi
# calcular el bono de cada vendedor
rdd_vendedor_bono = rdd_vendedor_dinero.map(lambda values: (values[0], values[1]*0.03))

# Muestro primer reduccion y ocurrencias
print(f'Primer elemento del reducer: <{rdd_vendedor_bono.first()}>')
print(f'Ocurrencias en reducer: {rdd_vendedor_bono.count()}.')
print()

Primer linea del archivo: ['17493', '6012', '21', '207.509827822219']
Ocurrencias en archivo: 119582.

Primer elemento del mapper: <('100', 341.811971935356)>
Ocurrencias en mapper: 119582.

Primer elemento del reducer: <('100', 286.54551868222)>
Ocurrencias en reducer: 15522.



Por último se muestra todo el contenido del RDD. Es decir, todos los pares ``<id_vendedor, bono>``:

In [ ]:
# Muestro todos los id_vendedor con su correspondiente bono
for vendedor in rdd_vendedor_bono.collect():
    print(vendedor)


___


A continuación se realiza el punto b:

In [70]:
# Reutilizo el rdd del punto anterior 
# Mapeo <[id_coordinador, id_vendedor], cantidad_productos_vendidos> 
# Ademas, convierto cantidades a integer y ordeno por clave
rdd_coord_ventas = rdd_ventas.map(lambda values: ((values[1], values[0]), int(values[2]))).sortByKey()

# Muestro el primer mapeo
print(f'Primer elemento del mapper: <{rdd_coord_ventas.first()}>')
print(f'Ocurrencias en mapper: {rdd_coord_ventas.count()}.')
print()

# Acumulo las cantidades por clave
rdd_coord_cantidades = rdd_coord_ventas.reduceByKey(lambda coord, cant: coord  +cant)

# Muestro la primer reduccion
print(f'Primer elemento del reducer: <{rdd_coord_cantidades.first()}>')
print(f'Ocurrencias en reducer: {rdd_coord_cantidades.count()}.')
print()

Primer elemento del mapper: <(('10008', '11947'), 50)>
Ocurrencias en mapper: 119582.

Primer elemento del reducer: <(('10008', '11947'), 292)>
Ocurrencias en reducer: 15522.



In [ ]:
# Muestro todo el contenido del rdd reducido
for val in rdd_coord_cantidades.collect():
    print(val)

In [72]:
# Finalizo la aplicacion
sc.stop()